<a href="https://colab.research.google.com/github/ErelAmit/bridge-bidding-bot/blob/main/bridge_bidding_full_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Basic setup: clone your repo and install required Python packages

import os, shutil

# Work in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# Remove any old clone to avoid conflicts
if os.path.exists("bridge-bidding-bot"):
    print("Removing existing bridge-bidding-bot folder...")
    shutil.rmtree("bridge-bidding-bot")

# Clone from GitHub
!git clone https://github.com/ErelAmit/bridge-bidding-bot.git
%cd /content/bridge-bidding-bot

# Install only what this project needs
!pip install -q scipy matplotlib

import torch, numpy as np
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA device:", torch.cuda.get_device_name(0))
print("NumPy version:", np.__version__)

Current directory: /content
Cloning into 'bridge-bidding-bot'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 94 (delta 33), reused 56 (delta 15), pack-reused 0 (from 0)
Receiving objects: 100% (94/94), 12.11 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/bridge-bidding-bot
Torch version: 2.9.0+cu126
CUDA available: True
CUDA device: NVIDIA L4
NumPy version: 2.0.2


In [2]:
# 2. Data upload helper
# This prints the path where your .mat files should live and lists any that already exist.

from pathlib import Path

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)
print("Data directory:", data_dir.resolve())
print("Currently present .mat files:")
for p in sorted(data_dir.glob("*.mat")):
    print("  ", p.name)

print("\nIf the six .mat files are not here yet,")
print("you will need to manually add them; this should not happen if using Google Colab")

Data directory: /content/bridge-bidding-bot/data
Currently present .mat files:
   cost_test.mat
   cost_train.mat
   cost_validate.mat
   data_test.mat
   data_train.mat
   data_validate.mat

If the six .mat files are not here yet,
you will need to manually add them; this should not happen if using Google Colab


In [3]:
# 3. Train the supervised MLP baseline (cross entropy, accuracy focused)
# This uses src/train_supervised.py and should detect CUDA if available
!python -m src.train_supervised

Project src:  /content/bridge-bidding-bot/src
Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Using device: cuda

Epoch 1/20
  Train loss: 1.4566, acc: 0.5180
  Val   loss: 0.9485, acc: 0.6500
  LR:         0.001000
  New best model saved to /content/bridge-bidding-bot/models/mlp_baseline.pt

Epoch 2/20
  Train loss: 0.9909, acc: 0.6358
  Val   loss: 0.8672, acc: 0.6688
  LR:         0.001000
  New best model saved to /content/bridge-bidding-bot/models/mlp_baseline.pt

Epoch 3/20
  Train loss: 0.9190, acc: 0.6579
  Val   loss: 0.8426, acc: 0.6810
  LR:         0.001000
  New best model saved to /content/bridge-bidding-bot/models/mlp_baseline.pt

Epoch 4/20
  Train loss: 0.8774, acc: 0.6736
  Val   loss: 0.8113, acc: 0.6895
  LR:         0.001000
  New best model saved to /content/bridge-bidding-bot/models/mlp_baseline.pt

Epoch 5/20
  Train loss: 0.8493, acc: 0.6816
  Val   loss: 0.7921, acc: 0.6969
  LR:         0.001000
  New best model saved 

In [4]:
# 4. Evaluate the MLP baseline and plot training curves

# Train vs val evaluation
!python -m src.eval_train_val

# Test set evaluation for top 1 and top 3 accuracy
!python -m src.eval_test_mlp

# Draw training curves (loss and accuracy) into docs folder
!python -m src.plot_training_curves

Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Checkpoint:   /content/bridge-bidding-bot/models/mlp_baseline.pt
Train size: 100000
Val size:   20000
Device: cuda

Eval mode metrics (no dropout, BN in eval):
  Train loss: 0.5673, acc: 0.7861
  Val   loss: 0.7255, acc: 0.7224
Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Checkpoint:   /content/bridge-bidding-bot/models/mlp_baseline.pt
Test size: 20000
Device: cuda

Test set metrics (MLP, eval mode):
  Loss:           1.0526
  Top 1 accuracy: 0.6301
  Top 3 accuracy: 0.9179
Wrote loss curves to: /content/bridge-bidding-bot/docs/mlp_loss_curves.png
Wrote accuracy curves to: /content/bridge-bidding-bot/docs/mlp_acc_curves.png


In [5]:
# 5. Hyperparameter and optimizer sweep for MLP (AdamW vs SGD, dropout, BN)

!python -m src.run_sweep

Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Results CSV:  /content/bridge-bidding-bot/docs/sweep_results.csv
Using device: cuda

=== Experiment ===
{'optimizer': 'adamw', 'hidden_dim': 256, 'num_layers': 3, 'dropout': 0.0, 'use_batchnorm': False, 'learning_rate': 0.001, 'weight_decay': 0.0001, 'max_grad_norm': 1.0, 'num_epochs': 20, 'batch_size': 512}

Epoch 1/20
  Train loss: 1.6365, acc: 0.4656
  Val   loss: 0.9930, acc: 0.6472

Epoch 2/20
  Train loss: 0.9601, acc: 0.6506
  Val   loss: 0.9701, acc: 0.6315

Epoch 3/20
  Train loss: 0.9058, acc: 0.6627
  Val   loss: 0.9030, acc: 0.6645

Epoch 4/20
  Train loss: 0.8629, acc: 0.6758
  Val   loss: 0.8595, acc: 0.6743

Epoch 5/20
  Train loss: 0.8275, acc: 0.6858
  Val   loss: 0.8357, acc: 0.6763

Epoch 6/20
  Train loss: 0.7990, acc: 0.6973
  Val   loss: 0.8107, acc: 0.6897

Epoch 7/20
  Train loss: 0.7775, acc: 0.7022
  Val   loss: 0.8132, acc: 0.6874

Epoch 8/20
  Train loss: 0.7576, acc: 0.

In [6]:
# 6. Train the attention plus RNN model using IMP based training
!python -m src.train_imps_hybrid

Project src:  /content/bridge-bidding-bot/src
Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Models dir:   /content/bridge-bidding-bot/models
Docs dir:     /content/bridge-bidding-bot/docs
Using device: cuda

Epoch 1/20
  Train expected cost: 0.2082
  Train best cost:     0.0391
  Train regret:        0.1691
  Train acc vs best:   0.1884
  Val   expected cost: 0.1916
  Val   best cost:     0.0391
  Val   regret:        0.1525
  Val   acc vs best:   0.2001
  New best model by regret saved to /content/bridge-bidding-bot/models/attn_rnn_imps.pt

Epoch 2/20
  Train expected cost: 0.1915
  Train best cost:     0.0391
  Train regret:        0.1525
  Train acc vs best:   0.2017
  Val   expected cost: 0.1916
  Val   best cost:     0.0391
  Val   regret:        0.1525
  Val   acc vs best:   0.2001
  New best model by regret saved to /content/bridge-bidding-bot/models/attn_rnn_imps.pt

Epoch 3/20
  Train expected cost: 0.1915
  Train best cost:     0.039

In [7]:
# 7. Evaluate the attention plus RNN model on test IMP metrics and plot its training curves

# Test IMP metrics
!python -m src.eval_imps_attn_rnn

# Training curves: expected IMP cost and accuracy vs best
!python -m src.plot_attn_rnn_imps_curves

Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Checkpoint:   /content/bridge-bidding-bot/models/attn_rnn_imps.pt
Test size: 20000
Device: cuda

Test set IMP metrics (attention plus RNN):
  Avg expected cost: 0.1997
  Avg best cost:     0.0000
  Avg regret:        0.1997
  Acc vs best:       0.1829
Wrote loss curves to: /content/bridge-bidding-bot/docs/attn_rnn_imps_loss_curves.png
Wrote accuracy curves to: /content/bridge-bidding-bot/docs/attn_rnn_imps_acc_curves.png


In [8]:
# 8. Evaluate the MLP baseline on the same IMP based metrics

!python -m src.eval_imps_mlp

Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Checkpoint:   /content/bridge-bidding-bot/models/mlp_baseline.pt
Test size: 20000
Device: cuda

Test set IMP metrics (MLP trained on accuracy):
  Avg expected cost: 0.0957
  Avg best cost:     0.0000
  Avg regret:        0.0957
  Acc vs best:       0.6109


In [8]:
# 9. Side by side comparison: MLP vs Attention plus RNN on IMP test metrics

!python -m src.compare_models

Project root: /content/bridge-bidding-bot
Data root:    /content/bridge-bidding-bot/data
Models dir:   /content/bridge-bidding-bot/models
MLP ckpt:     /content/bridge-bidding-bot/models/mlp_baseline.pt
AttnRNN ckpt: /content/bridge-bidding-bot/models/attn_rnn_imps.pt
Train size: 100000
Test size:  20000
Device: cuda

=== IMP comparison on test set ===
Model                            Avg expected cost   Avg best cost    Avg regret   Acc vs best
----------------------------------------------------------------------------------------------
Random uniform                              0.4772          0.0000        0.4772        0.0521
Majority best action                        0.1997          0.0000        0.1997        0.1829
MLP (CE trained)                            0.0871          0.0000        0.0871        0.6301
Attn+RNN (IMP trained)                      0.1997          0.0000        0.1997        0.1829
